In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train')
valid_pack = mz.datasets.wiki_qa.load_data('dev')
predict_pack = mz.datasets.wiki_qa.load_data('test')

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2117/2117 [00:00<00:00, 8485.51it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18828/18828 [00:04<00:00, 4606.35it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404539/404539 [00:00<00:00, 2718033.42it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2117/2117 [00:00<00:00, 8664.07it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18828/18828 [00:04<00:00, 4658.58it/s]
Processing text_right with transform: 100%|██████████| 18828/18828 [00:00<00:00, 84476.81it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 296/296 [00:00<00:00, 7579.77it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2708/2708 [00:00<00:00, 4710.58it/s]
Processing text_right with transform: 100%|██████████| 2708/2708 [00:00<00:00, 90772.45it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 630/630 [00:00<00:00, 8343.97it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 5914/5914 [00:01<00:00, 4341.16it/s]
Processing text_right with transform: 100%|██████████| 5914/5914 [00:00<00:00, 92137.99it/s]


In [6]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [7]:
model = mz.models.DRMMTKSModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['top_k'] = 20
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 50
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adagrad'
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to DRMMTKSModel.
Parameter "embedding_trainable" set to True.


In [8]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=50)

In [9]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [10]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = model.EvaluateOnCall(model, x=pred_x, y=pred_y, batch_size=20)
len(pred_y)

6116

In [12]:
model.fit_generator(train_generator, epochs=100, callbacks=[evaluate], workers=10)

Epoch 1/100


/data/users/fyx/NTMC-Community/MatchZoo/matchzoo/data_pack/data_pack.py:167: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  relation = self._relation.loc[index].reset_index(drop=True)


  4/160 [..............................] - ETA: 38s - loss: 0.9536 

StopIteration: cannot index with vector containing NA / NaN values

In [ ]:
model.predict(predict_pack_processed[:10].unpack()[0])

In [12]:
model.evaluate(*valid_pack_processed.unpack())

 128/2733 [>.............................] - ETA: 0s

InvalidArgumentError: Incompatible shapes: [22,1] vs. [23,1]
	 [[{{node loss/dense_4_loss/sub}} = Sub[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss/dense_4_loss/add, loss/dense_4_loss/lambda_3/strided_slice)]]

In [8]:
from keras.utils import plot_model
plot_model(model.backend, to_file='model.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.